<a href="https://colab.research.google.com/github/straxFromIbr/NLP_with_BERT/blob/main/Section5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pip && pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 >/dev/null
import torch
from transformers import BertJapaneseTokenizer, BertModel
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"


     |████████████████████████████████| 1.7 MB 13.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
